# Food Vision Big Classifier

**Introduction**

This project focuses on developing a machine learning model for classifying types of food from photographs. The goal is to build a robust image classification system capable of accurately identifying different food categories based on visual features. To train and evaluate the model, we utilized the Food-101 dataset, a comprehensive collection of food images sourced from Kaggle. This dataset comprises 101 distinct classes of food, providing a diverse and challenging benchmark for training and testing the model's performance. 

*Download data here:* https://www.kaggle.com/datasets/dansbecker/food-101